In [1]:
import numpy as np
import pandas as pd
import ast
import re

from src.utils import clean_meta_data, clean_keywords, clean_movies

In [2]:
metaData = clean_meta_data(pd.read_csv("../../data/movies_metadata.csv", low_memory=False))
keywords = clean_keywords(pd.read_csv("../../data/keywords.csv", low_memory=False))
users = pd.read_csv("../../data/users.dat", sep="::",
                    names=["UserId", "Gender", "AgeGroup", "Occupation", "Zip"])
movies = clean_movies(pd.read_csv("../../data/movies.dat", sep="::",
                     names=["MovieId", "Title", "Genres"]))

<ipython-input-2-1b5d3f23966c>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  users = pd.read_csv("../../data/users.dat", sep="::",
<ipython-input-2-1b5d3f23966c>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = clean_movies(pd.read_csv("../../data/movies.dat", sep="::",


In [4]:
metaData.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,Toy Story Collection,30000000,"Animation, Comedy, Family",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,English,Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"Adventure, Fantasy, Family",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"English, Français",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,Grumpy Old Men Collection,0,"Romance, Comedy",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,English,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"Comedy, Drama, Romance",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,English,Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,Father of the Bride Collection,0,Comedy,NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,English,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [5]:
keywords.head()

,id,keywords
0,862,"jealousy, toy, boy, friendship, friends, rival..."
1,8844,"board game, disappearance, based on children's..."
2,15602,"fishing, best friend, duringcreditsstinger, ol..."
3,31357,"based on novel, interracial relationship, sing..."
4,11862,"baby, midlife crisis, confidence, aging, daugh..."


In [6]:
users.head()

,UserId,Gender,AgeGroup,Occupation,Zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [7]:
movies.head()

,MovieId,Title,Genres,Release
0,1,Toy Story,"Animation, Children's, Comedy",1995
1,2,Jumanji,"Adventure, Children's, Fantasy",1995
2,3,Grumpier Old Men,"Comedy, Romance",1995
3,4,Waiting to Exhale,"Comedy, Drama",1995
4,5,Father of the Bride Part II,Comedy,1995
